In [1]:
import json
import torchaudio
from pathlib import Path
from copy import copy
from docx.api import Document
import soundfile as sf
import librosa
from pydub.utils import mediainfo
from tqdm.auto import tqdm



In [2]:
from data_utils import NeutuneSet

In [3]:
# dataset = NeutuneSet('/home/teo/userdata/nia_dataset/field_recording')
dataset = NeutuneSet('/home/teo/label-studio_files/nia_dataset/1cycle')

In [4]:
from pandas import read_table


class MetaCreator:
  def __init__(self, vocab_path):
    with open(vocab_path, 'r') as f:
      self.vocab = json.load(f)
    self.error_list = []

  def get_recording_type(self, wav_path):
    if '폴리' in wav_path:
      return '폴리'
    elif '합성' in wav_path:
      return '합성'
    elif '필드' in wav_path:
      return '필드 레코딩'

  def check_mp3_format(self, wav_path):
    mp3_path = wav_path.parent.parent / 'mp3' / wav_path.with_suffix('.mp3').name
    mp3_info = mediainfo(mp3_path)
    if int(mp3_info["sample_rate"]) != 48000:
      print(f"Error in MP3 SR: {mp3_path} is {mp3_info['sample_rate']}")
    if abs(int(mp3_info["bit_rate"]) - 320000) > 300 :
      print(f"Error in MP3 bit_rate: {mp3_path} is {mp3_info['bit_rate']}")
    return float(mp3_info["duration"])

  def check_name_is_wrong(self, wav_path):
    name = wav_path.stem
    high, middle, fine = [int(x) for x in name.split('-')[:3]]
    voc = self.vocab[fine-1]
    assert voc["소분류_번호"] == fine
    if high != voc['대분류_번호']:
      print(f"Correct High: {voc['대분류_번호']} / Current High: {high} / Current Fine: {fine}")
      return True
    if middle != voc['중분류_번호']:
      print(f"Correct middle: {voc['중분류_번호']} / Current middle: {middle} / Current Fine: {fine}")

      return True
    return False

  def add_error(self, wav_path, error_str):
    self.error_list.append({'file': wav_path.name, 'error': error_str})

  def create_meta_for_wav(self, wav_path):
    if isinstance(wav_path, str):
      wav_path = Path(wav_path)
    category_idx = int(wav_path.stem.split('-')[2]) - 1
    
    # try:
    #   mp3_dur = self.check_mp3_format(wav_path)
    # except:
    #   print(f"Error occured while reading mp3 of {wav_path}")


    if self.check_name_is_wrong(wav_path):
      # print(f"Name is wrong for {wav_path}")
      self.error_list.append(wav_path, "Category is wrong")
    meta = copy(self.vocab[category_idx])
    meta['녹음 방식'] = self.get_recording_type(str(wav_path))

    # y, sr = torchaudio.load(wav_path)
    ob = sf.SoundFile(wav_path)
    # if abs(mp3_dur - ob.frames / ob.samplerate) > 0.2:
    #   print(f"Duration of MP3 and Wav is different: {ob.frames / ob.samplerate}, {mp3_dur}")

    meta['샘플링 레이트'] = ob.samplerate
    meta['채널 수'] = ob.channels
    meta['Bit_depth'] = int(ob.subtype.split('_')[1])
    if meta['Bit_depth'] != 24:
      # print(f"Bit-depth of {wav_path} is {meta['Bit_depth']}")
      self.add_error(wav_path, f"Bit-depth is {meta['Bit_depth']}")
    if meta['채널 수'] != 2:
      print(f"Num Channels of {wav_path} is {meta['채널 수']}")
      self.add_error(wav_path, f"Num channels is {meta['채널 수']}")
    if meta['샘플링 레이트'] != 96000:
      print(f"Sampling rate of {wav_path} is {meta['샘플링 레이트']}")
      self.add_error(wav_path, f"Sampling rate is {meta['샘플링 레이트']}")

    meta['길이'] = ob.frames / ob.samplerate
    if True: #if it is field recording
      docx_path = wav_path.parent.parent / 'metadata' / wav_path.with_suffix('.docx').name
      if docx_path.exists():
        try:
          detailed_meta = self.read_table_from_docx(docx_path)
          if meta['녹음 방식'] == "필드 레코딩":
            meta["필드 레코딩 메타"] = detailed_meta
          meta["제목"] = detailed_meta["제목"]
        except Exception as e:
          self.add_error(wav_path, f"Error occured while handling the corresponding docx")
      else:
        self.add_error(wav_path, "Meta docx does not exist")

    # self.get_audio_features(ob)
    return meta

  def read_table_from_docx(self, docx_path):
    document = Document(docx_path)
    table = document.tables[0]
    table_dict = {row.cells[0].text:row.cells[1].text for row in table.rows[1:]}

    return table_dict


  def get_audio_features(self, soundfile:sf.SoundFile):
    y = soundfile.read()
    y = y.mean(1)
    spec_centroid = librosa.feature.spectral_centroid(y = y, sr=soundfile.samplerate)
    print(spec_centroid)

    return

meta_creator = MetaCreator('vocab.json')
# meta_creator.create_meta_for_wav(dataset.wav_list[0])
entire_meta = [meta_creator.create_meta_for_wav(wav) for wav in tqdm(dataset.wav_list)]

  0%|          | 0/624 [00:00<?, ?it/s]

In [16]:
len(dataset.wav_list)

475

In [34]:
[error['file'] for error in meta_creator.error_list if error['error'] == 'Error occured while handling the corresponding docx']

['2-4-78-1-윤정현.wav',
 '2-4-78-1-조현진.wav',
 '2-4-78-2-윤정현.wav',
 '2-4-78-2-조현진.wav',
 '2-4-78-3-윤정현.wav',
 '2-4-78-3-조현진.wav',
 '2-4-79-1-윤정현.wav',
 '2-4-79-1-조현진.wav',
 '2-4-79-2-윤정현.wav',
 '2-4-79-2-조현진.wav',
 '2-4-79-3-윤정현.wav',
 '2-4-79-3-조현진.wav',
 '2-4-79-4-윤정현.wav',
 '2-4-79-4-조현진.wav',
 '2-4-80-1-윤정현.wav',
 '2-4-80-1-조현진.wav',
 '2-4-80-2-윤정현.wav',
 '2-4-80-2-조현진.wav',
 '2-4-80-3-윤정현.wav',
 '2-4-80-3-조현진.wav',
 '2-4-92-1-윤정현.wav',
 '2-4-92-1-조현진.wav',
 '2-4-92-2-윤정현.wav',
 '2-4-92-2-조현진.wav',
 '2-4-92-3-윤정현.wav',
 '2-4-92-3-조현진.wav',
 '2-4-92-4-윤정현.wav',
 '1-1-1-1-김시온.wav',
 '1-1-1-13-박채원.wav',
 '1-1-1-14-박채원.wav',
 '1-1-1-15-박채원.wav',
 '1-1-1-2-허진만.wav',
 '1-1-1-3-김민규.wav',
 '1-1-1-3-김시온.wav',
 '1-1-1-4-김민규.wav',
 '1-1-2-1-김민규.wav',
 '1-1-2-1-김시온.wav',
 '1-1-2-1-김영운.wav',
 '1-1-2-1-윤상빈.wav',
 '1-1-2-1-이주용.wav',
 '1-1-2-1-정욱.wav',
 '1-1-2-1-허진만.wav',
 '1-1-2-10-김민규.wav',
 '1-1-2-11-김민규.wav',
 '1-1-2-12-김민규.wav',
 '1-1-2-13-김민규.wav',
 '1-1-2-2-김민규.wav',
 '1-1-2-2-김시온.wav',
 '1-1-2

In [5]:
meta_creator.error_list

[{'file': '2-4-78-1-어상진.wav', 'error': 'Meta docx does not exist'},
 {'file': '2-4-78-1-윤정현.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': '2-4-78-1-조현진.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': '2-4-78-2-어상진.wav', 'error': 'Meta docx does not exist'},
 {'file': '2-4-78-2-윤정현.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': '2-4-78-2-조현진.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': '2-4-78-3-어상진.wav', 'error': 'Meta docx does not exist'},
 {'file': '2-4-78-3-윤정현.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': '2-4-78-3-조현진.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': '2-4-79-1-어상진.wav', 'error': 'Meta docx does not exist'},
 {'file': '2-4-79-1-윤정현.wav',
  'error': 'Error occured while handling the corresponding docx'},
 {'file': '2-4-79-1-조현진.wav',
  'error': 'Error o

In [27]:
entire_meta[300]

{'소분류': '병원',
 '소분류_번호': 2,
 '중분류': '실제 공간(앰비언스)',
 '대분류': '공간',
 '중분류_번호': 1,
 '대분류_번호': 1,
 '녹음 방식': '필드 레코딩',
 '샘플링 레이트': 96000,
 '채널 수': 2,
 'Bit_depth': 24,
 '길이': 120.0}

In [60]:
for meta in entire_meta:
  if '필드 레코딩 메타' in meta:
    print(meta['필드 레코딩 메타']['소리의 강도'])

-30 dB
-36 dB~-24db
-36 dB
-30 dB
-36dB~-24db
-48 dB
-36~-20 dB
-24dB
-30 dB
-30 dB
-33 dB
-30 dB
-33 dB
-30 dB
-30 dB
-33 dB
-20 dB
-20 dB
-20 dB
-20 dB
-20 dB
-27 dB
-20 dB
-20 dB
-20 dB
-27 dB
-24 dB
-27 dB
-27 dB
-30 dB
-30 dB
-24 dB
-30 dB
-20 dB
-44.4 db
-25.6 db
-25.6 db
-25.6 db
-25.6 db
-25.6 db
-25.6 db
-44.4 db
-42.7 db
-35.2 db
-25.6 db
-25.6 db
-25.6 db
-25.6 db
-25.6 db
-15dBFS
-5dBFS
-7dBFS
-9dBFS
-9dBFS
-9dBFS
-15dBFS
-14dBFS
-14dBFS
-12dBFS
-15dBFS
-20dBFS
-20dBFS
-5dBFS
-5dBFS


In [49]:
test = Document("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/metadata/1-2-5-1-김민규.docx")
table = test.tables[0]

for row in table.rows:
  print(row.cells[0].text, '//', row.cells[1].text)

1차 원시데이터 기록표 // 
1-2-5-1 // 제목
녹음자 성명 // 김민규
2022.07.12 // 사용 기자재
오디오 포맷 // 스테레오 - wav (96k, 24bit)
284초 // 녹음 장소
녹음 지점 // 37.482062, 126.942231
X-Ray 촬영기기로부터 약 1.5미터 // 기후 등 환경 조건
소리의 강도 // -30 dB
없음 // 기타 특이사항


IndexError: tuple index out of range

In [43]:
meta_creator.read_table_from_docx("/home/teo/userdata/nia_dataset/field_recording/4팀 김양우 1주차/metadata/1-1-3-1-한은광.docx")

IndexError: tuple index out of range

In [17]:
Path("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/metadata/1-2-7-6-백승렬.docx").exists()

True

In [11]:
table = meta_creator.read_table_from_docx('1-1-1-1-송영재.docx')
table

{'분류 번호': '1-1-1',
 '제목': '로얄호텔 서울 2층',
 '녹음자 성명': '송영재',
 '녹음 연/월/일': '2022.7.16',
 '사용 기자재': 'Zoom H6 레코더, 이어폰',
 '오디오 포맷': '스테레오 - wav (96k, 24bit) / mp3',
 '오디오 길이': '180초',
 '녹음 장소': '서울 중구 명동길 61 로얄호텔 서울',
 '녹음 지점': '37.564296, 126.985583',
 '음원으로부터의 거리': '약 3m',
 '기후 등 환경 조건': '실내',
 '소리의 강도': '-30 dB',
 '보안 이슈': '없음',
 '기타 특이사항': '-'}

In [15]:
meta_creator.vocab['idx2title']

['호텔',
 '병원',
 '놀이공원',
 '상업 몰',
 '공항',
 '전시장',
 '백화점',
 '마트',
 '재래시장',
 '지하상가',
 '주차장',
 '버스 정류장',
 '버스 터미널',
 '지하철역',
 '기차역',
 '레스토랑/음식점',
 '카페',
 '술집',
 '영화관',
 '헬스장',
 '상업지역',
 '도서관',
 '공원',
 '관공서',
 '사무실',
 '종교 시설',
 '수영장',
 '경기장 (실내)',
 '경기장 (실외)',
 '교외 거주지역',
 '아파트 단지',
 '가정 실내',
 '룸톤',
 '도심 지역',
 '교통 소리',
 '군중 소리',
 '동물원',
 '학교',
 '창고',
 '공장',
 '항구',
 '숲',
 '드넓은 초원',
 '동굴',
 '바닷가',
 '강/계곡/하천/냇가 등',
 '호숫가',
 '폭포',
 '가상 업무 공간',
 '고대 문명 도시',
 '해저 도시',
 '하늘 도시',
 '판타지 중세 성',
 '서부 총잡이 도시',
 '공상과학/사이버펑크 미래도시',
 '무한한 터널',
 '타임머신',
 '워프',
 '레이싱',
 '기계 도시',
 '우주 도시',
 '화산지대',
 '실험실',
 '우주선 복도',
 '우주 정거장',
 '서버실',
 '상황실',
 '신전',
 '광산',
 '외계 도시',
 '지하 도시',
 '우주 공간 시나리오',
 '우주선 탑승 시나리오',
 '자연 재해 시나리오',
 '인공 전쟁/전투 시나리오',
 '가상 전쟁/전투 시나리오',
 '위급 상황 관련 시나리오',
 '건축 관련 소리 시나리오',
 '음식 만드는 시나리오',
 '실내 청소 시나리오',
 '샤워 관련 소리 시나리오',
 '자전거 탑승 시나리오',
 '버스 탑승 시나리오',
 '차량 탑승 시나리오',
 '오토바이 탑승 시나리오',
 '비행기 탑승 시나리오',
 '헬리콥터 탑승 시나리오',
 '보트 탑승 시나리오',
 '유람선 탑승 시나리오',
 '기차 탑승 시나리오',
 '폭발 관련 소리 시나리오',
 '구강 발생 소리 시

In [67]:
mediainfo("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/mp3/1-2-7-6-백승렬.mp3")

{'index': '0',
 'codec_name': 'mp3',
 'codec_long_name': 'MP3 (MPEG audio layer 3)',
 'profile': 'unknown',
 'codec_type': 'audio',
 'codec_time_base': '1/48000',
 'codec_tag_string': '[0][0][0][0]',
 'codec_tag': '0x0000',
 'sample_fmt': 'fltp',
 'sample_rate': '48000',
 'channels': '2',
 'channel_layout': 'stereo',
 'bits_per_sample': '0',
 'id': 'N/A',
 'r_frame_rate': '0/0',
 'avg_frame_rate': '0/0',
 'time_base': '1/14112000',
 'start_pts': '324870',
 'start_time': '0.023021',
 'duration_ts': '2693246976',
 'duration': '190.848000',
 'bit_rate': '320132',
 'max_bit_rate': 'N/A',
 'bits_per_raw_sample': 'N/A',
 'nb_frames': 'N/A',
 'nb_read_frames': 'N/A',
 'nb_read_packets': 'N/A',
 'DISPOSITION': {'default': '0',
  'dub': '0',
  'original': '0',
  'comment': '0',
  'lyrics': '0',
  'karaoke': '0',
  'forced': '0',
  'hearing_impaired': '0',
  'visual_impaired': '0',
  'clean_effects': '0',
  'attached_pic': '0',
  'timed_thumbnails': '0'},
 'TAG': {'encoder': 'LAME3.100', 'TYER':

In [41]:
import docx

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

test = getText('1-1-1-1-송영재.docx')

In [42]:
from docx.api import Document

# Load the first table from your document. In your example file,
# there is only one table, so I just grab the first one.
document = Document('1-1-1-1-송영재.docx')
table = document.tables[0]

In [43]:
data = []

keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

    # Establish the mapping based on the first row
    # headers; these will become the keys of our dictionary
    if i == 0:
        keys = tuple(text)
        continue

    # Construct a dictionary for this row, mapping
    # keys to values for this row
    row_data = dict(zip(keys, text))
    data.append(row_data)
data[0]['']

'1-1-1'